In [15]:
# Import necessary libraries
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd

# Load the Titanic dataset
titanic_data = pd.read_csv('titanic.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    # Dropdown to select a feature
    dcc.Dropdown(
        id='feature-dropdown',
        options=[{'label': col, 'value': col} for col in titanic_data.columns],
        value='Age',  # default feature
        style={'width': '50%'}
    ),

    # Visualization components
    dcc.Graph(id='feature-plot'),
    dcc.Graph(id='survival-count-plot'),
    dcc.Graph(id='class-distribution-plot'),
    dcc.Graph(id='3d-scatter-plot'),
    dcc.Graph(id='survival-heatmap'),

    # Hidden div to store the data
    html.Div(id='titanic-data-store', style={'display': 'none'}),

    # Hidden div to store the selected feature
    html.Div(id='selected-feature-store', style={'display': 'none'})
])

# Callback to update the data store
@app.callback(
    Output('titanic-data-store', 'children'),
    [Input('feature-dropdown', 'value')]
)
def update_data_store(selected_feature):
    return titanic_data.to_json()

# Callback to update feature plot
@app.callback(
    Output('feature-plot', 'figure'),
    [Input('feature-dropdown', 'value')],
    [State('titanic-data-store', 'children')]
)
def update_feature_plot(selected_feature, titanic_data_json):
    if titanic_data_json is None:
        # Handle the case when data is not yet available
        return px.scatter(title='Waiting for data...', labels={'x': selected_feature})

    data = pd.read_json(titanic_data_json)
    return create_feature_plot(selected_feature, data)

def create_feature_plot(selected_feature, data):
    if selected_feature not in data.columns:
        raise ValueError(f"Selected feature '{selected_feature}' not found in the dataset.")

    if pd.api.types.is_numeric_dtype(data[selected_feature]):
        fig = px.histogram(data, x=selected_feature, color='Survived', barmode='overlay',
                           title=f'{selected_feature} Distribution by Survival')
    else:
        fig = px.bar(data, x=selected_feature, color='Survived',
                     title=f'{selected_feature} Count by Survival')

    return fig

# Callback to update survival count plot
@app.callback(
    Output('survival-count-plot', 'figure'),
    [Input('titanic-data-store', 'children')]
)
def update_survival_count_plot(titanic_data_json):
    if titanic_data_json is None:
        return px.bar(title='Waiting for data...', labels={'x': 'Survived'})

    data = pd.read_json(titanic_data_json)
    return create_survival_count_plot(data)

def create_survival_count_plot(data):
    fig = px.bar(data, x='Survived', title='Survival Count')
    return fig

# Callback to update class distribution plot
@app.callback(
    Output('class-distribution-plot', 'figure'),
    [Input('titanic-data-store', 'children')]
)
def update_class_distribution_plot(titanic_data_json):
    if titanic_data_json is None:
        return px.pie(title='Waiting for data...', labels={'names': 'Pclass'})

    data = pd.read_json(titanic_data_json)
    return create_class_distribution_plot(data)

def create_class_distribution_plot(data):
    fig = px.pie(data, names='Pclass', title='Class Distribution')
    return fig

# Callback to update 3D scatter plot
@app.callback(
    Output('3d-scatter-plot', 'figure'),
    [Input('titanic-data-store', 'children')]
)
def update_3d_scatter_plot(titanic_data_json):
    if titanic_data_json is None:
        return px.scatter_3d(title='Waiting for data...', labels={'x': 'Age', 'y': 'Fare', 'z': 'Pclass'})

    data = pd.read_json(titanic_data_json)
    return create_3d_scatter_plot(data)

def create_3d_scatter_plot(data):
    fig = px.scatter_3d(data, x='Age', y='Fare', z='Pclass', color='Survived',
                        title='3D Scatter Plot (Age, Fare, Pclass) by Survival')
    return fig

# Callback to update survival heatmap
@app.callback(
    Output('survival-heatmap', 'figure'),
    [Input('titanic-data-store', 'children')]
)
def update_survival_heatmap(titanic_data_json):
    if titanic_data_json is None:
        return px.imshow(title='Waiting for data...', labels=dict(color="Survival Correlation"))

    data = pd.read_json(titanic_data_json)
    return create_survival_heatmap(data)

def create_survival_heatmap(data):
    fig = px.imshow(data.corr(), labels=dict(color="Survival Correlation"),
                    title='Survival Correlation Heatmap')
    return fig

# Run the app
if __name__ == '__main__':
    # Use a different port, for example, 8051
    app.run_server(mode='inline', port=8051)



OSError: Address 'http://127.0.0.1:8051' already in use.
    Try passing a different port to run_server.